In [16]:
import numpy as np
from util import *
from gurobipy import *

In [17]:
nodes = 3
lines = 3
x = np.zeros([nodes,nodes])
x[0][1] = .1
x[0][2] = .1
x[1][2] = .1
x[1][0] = x[0][1]
x[2][0] = x[0][2]
x[2][1] = x[1][2]
x

array([[ 0. ,  0.1,  0.1],
       [ 0.1,  0. ,  0.1],
       [ 0.1,  0.1,  0. ]])

In [18]:
M = np.zeros([nodes,nodes])
for i in range(nodes):
    for j in range(nodes):
        if i != j:
            M[i,j] = -1./x[i,j]
        else:
            M[i,j] =sum( 1./x[k,j] for k in range(nodes) if k != j)
M

array([[ 20., -10., -10.],
       [-10.,  20., -10.],
       [-10., -10.,  20.]])

In [19]:
Z = np.linalg.inv(M[1::,1::])
Z

array([[ 0.06666667,  0.03333333],
       [ 0.03333333,  0.06666667]])

In [20]:
S = np.zeros([nodes, nodes, nodes])
for i in range(nodes):
    for l in range(nodes):
        for k in range(nodes):
            if k == 0 and l != 0:
                S[k,l,i] = -1*Z[l-1,i-1]
            elif k!= 0 and l==0:
                S[k,l,i] = Z[k-1,i-1]
            elif k!= 0 and l!=0 and k!=l:
                S[k,l,i] = Z[k-1,i-1] - Z[l-1,i-1]
S

array([[[ 0.        ,  0.        ,  0.        ],
        [-0.03333333, -0.06666667, -0.03333333],
        [-0.06666667, -0.03333333, -0.06666667]],

       [[ 0.03333333,  0.06666667,  0.03333333],
        [ 0.        ,  0.        ,  0.        ],
        [-0.03333333,  0.03333333, -0.03333333]],

       [[ 0.06666667,  0.03333333,  0.06666667],
        [ 0.03333333, -0.03333333,  0.03333333],
        [ 0.        ,  0.        ,  0.        ]]])

In [21]:
P = np.array([2,1,-3])
F = np.zeros([nodes,nodes])
for k,l in [(0,1), (0,2), (1,2), (1,0), (2,0), (2,1)]:
    F[k,l] = sum(P[i] * 1./x[k,l] * S[k,l,i] for i in range(nodes-1))
F

array([[ 0.        , -1.33333333, -1.66666667],
       [ 1.33333333,  0.        , -0.33333333],
       [ 1.66666667,  0.33333333,  0.        ]])

In [22]:

m = Model("Grid")

load = m.addVars(nodes, name="load", lb=-GRB.INFINITY)
m.setObjective(quicksum(load[i] for i in range(1,nodes)), GRB.MAXIMIZE)


lines = [(0,1), (0,2), (1,2)]
limits = np.ones([nodes,nodes]) * 10
F = {}
for k,l in lines:
    F[(k,l)] = m.addVar(name="F"+str((k,l)))
    m.addConstr(F[(k,l)] == sum(load[i] * 1./x[k,l] * S[k,l,i] for i in range(nodes-1)))
    m.addConstr(F[(k,l)] <= limits[k,l])
   
    
m.addConstr(quicksum(load) == 0)

print_lp(m)
solve_lp(m)

\ Model Grid
\ LP format - for model browsing. Use MPS format to capture full model detail.
Maximize
  load[1] + load[2]
Subject To
 R0: 0.3333333333333333 load[0] + 0.6666666666666666 load[1] + F(0,_1) = 0
 R1: F(0,_1) <= 10
 R2: 0.6666666666666666 load[0] + 0.3333333333333333 load[1] + F(0,_2) = 0
 R3: F(0,_2) <= 10
 R4: 0.3333333333333333 load[0] - 0.3333333333333333 load[1] + F(1,_2) = 0
 R5: F(1,_2) <= 10
 R6: load[0] + load[1] + load[2] = 0
Bounds
 load[0] free
 load[1] free
 load[2] free
End

Optimize a model with 7 rows, 6 columns and 15 nonzeros
Coefficient statistics:
  Matrix range     [3e-01, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 1e+01]
Presolve removed 7 rows and 6 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.0000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.03 seconds
Optimal obje